# Feature Engineering with cudf

In [1]:
from feature_engineering import feature_engineering
import cudf as dd

In [6]:
import gc

In [2]:
bureau_balance = dd.read_parquet('raw_data/bureau_balance.parquet')
bureau = dd.read_parquet('raw_data/bureau.parquet')
cc_balance = dd.read_parquet('raw_data/cc_balance.parquet')
payments = dd.read_parquet('raw_data/payments.parquet')
pc_balance = dd.read_parquet('raw_data/pc_balance.parquet')
prev = dd.read_parquet('raw_data/prev.parquet')
train = dd.read_parquet('raw_data/train.parquet')
test = dd.read_parquet('raw_data/test.parquet')

In [3]:
train.shape

(307511, 121)

In [5]:
test.shape

(48744, 120)

In [4]:
train_target = train['TARGET']
unified = dd.concat([train.drop('TARGET', axis=1), test])

In [11]:
unified.shape

(356255, 120)

In [7]:
del(train)
del(test)
gc.collect()

51

In [8]:
avg_bureau, sum_cc_balance, sum_payments, sum_pc_balance, sum_prev, unified_feat = \
    feature_engineering(bureau_balance, bureau, cc_balance, payments, pc_balance, 
                        prev, unified)

procecssing bureau balance
procecssing cc balance
procecssing bureau
procecssing payments
processing pc_balance
processing prev table
merging feats into train and test - part1
extra feats
feats done


In [20]:
del(avg_bureau)
del(sum_cc_balance)
del(sum_payments)
del(sum_pc_balance)
del(sum_prev)

In [23]:
train_feats = unified_feat.iloc[:307511].merge(train_target, how='left', 
                                               left_index=True, right_index=True)

In [22]:
test_feats = unified_feat.iloc[307511:]

In [26]:
train_feats[['TARGET', 'CODE_GENDER']].groupby('TARGET').count()

,CODE_GENDER
TARGET,
0,277216
1,24375


In [27]:
train_feats.to_parquet('data_eng/feats/train_feats.parquet')
test_feats.to_parquet('data_eng/feats/test_feats.parquet')

# Testing with cu-dask

In [ ]:
#cudf load
from dask_cuda import LocalCUDACluster
cluster=LocalCUDACluster()

NOTE: enter proxy/8787/status to get the dask dashboard - issue with kernels / conda / juypter

In [ ]:
from dask.distributed import Client
#import dask.dataframe as dd
client = Client(cluster)
client

In [ ]:
client.id

In [ ]:
client.shutdown()

## Temp Debug

In [ ]:
type(sum_prev)

### Loading our previous test data

In [ ]:
## reload our parquet
avg_bureau = dd.read_parquet('data_eng/avg_bureau')
sum_cc_balance = dd.read_parquet(path='data_eng/sum_cc_balance') # index
sum_payments = dd.read_parquet(path='data_eng/sum_payments')
sum_pc_balance = dd.read_parquet(path='data_eng/sum_pc_balance') # index
sum_prev = dd.read_parquet(path='data_eng/sum_prev') # index
train = dd.read_csv('data/application_train.csv')

In [ ]:
train.SK_ID_CURR.isnull().any().compute()

In [ ]:
# Merging the dataframes
## Lets try and merge everything with dask this time
train_feat_1 = train.drop('TARGET', axis=1) \
    .merge(sum_prev, how='left', left_on='SK_ID_CURR', right_index=True) #\
    #.drop('SK_ID_CURR', axis=1) 
    

In [ ]:
    .merge(avg_bureau, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR') \
    .merge(sum_cc_balance, how='left', left_on='SK_ID_CURR', right_index=True) #\
    .merge(sum_payments, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR') \
    .merge(sum_pc_balance, how='left', left_on='SK_ID_CURR', right_index=True) #\
    

In [ ]:
train_feat_1.SK_ID_CURR.isnull().any().compute()
#.head(10)

In [ ]:
train_feat_1.SK_ID_CURR.isnull().count().compute()
#train_feat_1.SK_ID_CURR.head(10)

In [ ]:
train_feat_1.head(10)

In [ ]:
client.shutdown()